### Hyper Parameters Tuning for NN4 Model (Conducted on Google Colab)

In [ ]:
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
X_train, X_validation, y_train, y_validation = train_test_split(X_tv, y_tv, test_size=0.25,random_state=random_state)

undersampler = RandomUnderSampler(sampling_strategy='majority')
cv = KFold(n_splits=5,random_state=random_state,shuffle=True)
collection = []
important_dict = {}

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
def creating_nn4(hp):
    model = Sequential()

    unit1 = hp.Int('lay1',min_value=1, max_value = 50, step = 1)
    unit2 = hp.Int('lay2',min_value=1, max_value = 50, step = 1)
    unit3 = hp.Int('lay3',min_value=1, max_value = 50, step = 1)
    unit4 = hp.Int('lay4',min_value=1, max_value = 50, step = 1)
    unit5 = hp.Float('dropout1', min_value=0, max_value=0.6, step=0.1)
    unit6 = hp.Float('dropout2', min_value=0, max_value=0.6, step=0.1)
    unit7 = hp.Float('dropout3', min_value=0, max_value=0.6, step=0.1)
    unit8 = hp.Float('dropout4', min_value=0, max_value=0.6, step=0.1)
    unit9 = hp.Float('dropout5', min_value=0, max_value=0.6, step=0.1)
    unit10 = hp.Choice('regularization', values = [1e-1, 1e-2, 1e-3])
    learning_rate = hp.Choice('learning_rate',values=[0.001,0.0001])

    model.add(Dense(42, activation='tanh', kernel_regularizer=regularizers.l2(unit10), bias_regularizer=regularizers.l2(unit10), input_shape=(42,)))
    model.add(Dropout(unit5))
    model.add(Dense(unit1,activation='tanh', kernel_regularizer=regularizers.l2(unit10), bias_regularizer=regularizers.l2(unit10)))
    model.add(Dropout(unit6))
    model.add(Dense(unit2,activation='tanh', kernel_regularizer=regularizers.l2(unit10), bias_regularizer=regularizers.l2(unit10)))
    model.add(Dropout(unit7))
    model.add(Dense(unit3,activation='tanh', kernel_regularizer=regularizers.l2(unit10), bias_regularizer=regularizers.l2(unit10)))
    model.add(Dropout(unit8))
    model.add(Dense(unit4,activation='tanh', kernel_regularizer=regularizers.l2(unit10), bias_regularizer=regularizers.l2(unit10)))
    model.add(Dropout(unit9))
    model.add(Dense(1, activation='sigmoid'))


    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),loss=BinaryCrossentropy(from_logits=False),
                  metrics=[f1, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

In [ ]:
for fold_train_index, fold_test_index in cv.split(X_train):
    X_train_fold = X_train[fold_train_index]
    X_test_fold = X_train[fold_test_index]
    y_train_fold = y_train[fold_train_index]
    y_test_fold = y_train[fold_test_index]

    X_train_fold, y_train_fold = undersampler.fit_resample(X_train_fold, y_train_fold)
    tuner_nn4 = kt.Hyperband(creating_nn4,objective=kt.Objective("val_f1", direction="max"), max_epochs=100)
    tuner_nn4.search(X_train_fold, y_train_fold, epochs=50, validation_data=[X_validation, y_validation], shuffle=True,
                     callbacks=[ClearTrainingOutput()], batch_size=100)

In [ ]:
"""
|Best Value        |hyperparameters
|22                |lay1
|7                 |lay2
|48                |lay3
|50                |lay4
|0                 |dropout1
|0.2               |dropout2
|0.4               |dropout3
|0.1               |dropout4
|0.3               |dropout5
|0.001             |regularization
|0.001             |learning_rate
|100               |tuner/epochs
|34                |tuner/initial_epoch
|3                 |tuner/bracket
|3                 |tuner/round
|0204              |tuner/trial_id
|0.5259224772453308|F1 Score
"""